# ETL Notebook

This notebook is to do some DataFrame visualisation for our ETL process. We will run connect to the Database and run our SQL queries to extract our defined dimensions.

This can also be our testbed for potential transforms that we will be conducting for our NLP models.

In [31]:
# Import libraries
import mysql.connector
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from uuid import uuid4
load_dotenv()

True

In [32]:
db_host = os.getenv("DATABASE_ENDPOINT")
db_user = os.getenv("DATABASE_USERNAME")
db_pw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
db_port = os.getenv("DATABASE_PORT")

dwh_host = os.getenv("DATAWH_ENDPOINT")
dwh_user = os.getenv("DATAWH_USERNAME")
dwh_pw = os.getenv("DATAWH_PASSWORD")
dwh_name = os.getenv("DATAWH_NAME")
dwh_port = os.getenv("DATAWH_PORT")


db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

engine = create_engine(f'mysql://{dwh_user}:{dwh_pw}@{dwh_host}:{dwh_port}/{dwh_name}', echo=False)

dwh = engine.connect()

In [33]:
# Check tripadvisor_reviews table

str_sql = '''
SELECT *
FROM tripadvisor_reviews
'''

df = pd.read_sql(sql=str_sql, con=db_tripadvisor)
df

/var/folders/0m/1wpwx38n6tdb_9y7mx9pvb_c0000gn/T/ipykernel_46333/3910135080.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=str_sql, con=db_tripadvisor)


,Review Title,Review Text,Date of Stay,Author Contribution,Rating
0,Must see in Singapore,A must not miss place for tourists to visit wh...,2024-03-01,73,5.0
1,Marina Bay world class,Amazing hotel and loved the facilities. Being ...,2024-03-01,1,5.0
2,Nice touch.,While the initial check in experience was not ...,2024-03-01,2,5.0
3,Amazing experience at MBS with superb hospital...,I booked the Sands Premiere Suite to celebrate...,2024-03-01,1,5.0
4,Incredibile,"Nina is amazing as all the team, I loved her h...",2024-03-01,2,5.0
...,...,...,...,...,...
11227,Aweseome view over the Harbour,Amazing design ! I wouldnt recommend to sleep...,2014-03-01,131,4.0
11228,Excellent Property but poor services,Stayed in this hotel during the period Novembe...,2014-03-01,15,3.0
11229,Stunning &amp; Mesmeric,When you walk out on to that roof for the firs...,2014-03-01,25,5.0
11230,A Slice of Heaven,One can say what they want about this property...,2014-02-01,5,4.0


In [51]:
# Time dimension
# TimeID
# StayDate
# StayDateYear
# StayDateMonth
# StayDateDay
# StayDateDayOfWeek
# StayDateWeek

time_sql = """
SELECT `Date of Stay` as StayDate, YEAR(`Date of Stay`) AS StayDateYear, MONTH(`Date of Stay`) AS StayDateMonth, Day(`Date of Stay`) AS StayDateDay, IF((DayOfWeek(`Date of Stay`) - 1) = 0, 7, DayOfWeek(`Date of Stay`) - 1) As StayDateDayOfWeek, WEEK(`Date of Stay`) AS StayDateWeek
FROM tripadvisor_reviews;
"""

df = pd.read_sql(sql=time_sql, con=db_tripadvisor)
df['TimeID'] = df['StayDate'].apply(lambda x: uuid4())

/var/folders/0m/1wpwx38n6tdb_9y7mx9pvb_c0000gn/T/ipykernel_46333/89238451.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=time_sql, con=db_tripadvisor)


,StayDate,StayDateYear,StayDateMonth,StayDateDay,StayDateDayOfWeek,StayDateWeek,TimeID
0,2024-03-01,2024,3,1,5,8,9fb6f494-2ba5-4240-a5a7-50037d697d7a
1,2024-03-01,2024,3,1,5,8,e215bce8-69f8-4ef9-8afb-a79a49a7347d
2,2024-03-01,2024,3,1,5,8,3d87fe2f-8336-4112-8e63-3c1b3de11d52
3,2024-03-01,2024,3,1,5,8,441f7ef9-655d-458e-8452-56e53d172af8
4,2024-03-01,2024,3,1,5,8,83ded0c7-d502-45c4-8bc7-e037c809db39
...,...,...,...,...,...,...,...
11227,2014-03-01,2014,3,1,6,8,33807382-526e-405b-8960-132f80ce5fcc
11228,2014-03-01,2014,3,1,6,8,6e84ba2e-3323-48c2-b3a9-9662a472002d
11229,2014-03-01,2014,3,1,6,8,38a85c19-9883-4954-841a-b895d3803b71
11230,2014-02-01,2014,2,1,6,4,1b3e48e0-daad-455a-ba8f-11b5f4708a25


In [ ]:
# Review Dimension
# ReviewID
# ReviewText
# ReviewTitle

